In [1]:
import fastbook
fastbook.setup_book()

In [2]:
from fastai.vision.all import *
from fastbook import *

In [12]:
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()

path = untar_data('https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz')

In [4]:
train_path = (path/'training').ls().sorted()
train_x = []
train_y = []
for i, pat in enumerate(train_path):
    num = [tensor(Image.open(o)).float() for o in (pat).ls()]
    train_x += num
    train_y += tensor([i]*len(num)).unsqueeze(1)
# print(train_x[0].shape, train_x[0].view(-1, 28*28))
train_x = [x.view(-1, 28*28) for x in train_x]
training_dset = list(zip(train_x, train_y))
test_x = []
test_y = []
for i, pat in enumerate((path/'testing').ls().sorted()):
    num = [tensor(Image.open(o)).float() for o in (pat).ls()]
    test_x += num
    test_y += tensor([i]*len(num)).unsqueeze(1)
test_x = [x.view(-1, 28*28) for x in test_x]   
testing_dset = list(zip(test_x, test_y))

In [5]:
training_dl = DataLoader(training_dset, batch_size=256)
testing_dl = DataLoader(testing_dset, batch_size=256)

In [6]:
dls = DataLoaders(training_dl, testing_dl)

In [7]:
x = first(dls)

In [8]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,10)
)

In [9]:
# def batch_accuracy(xb, yb):
#     preds = xb.sigmoid()
#     correct = (preds>0.5) == yb
#     return correct.float().mean()
# def mnist_loss(predictions, targets):
#     predictions = predictions.sigmoid()
#     return torch.where(targets==1, 1-predictions, predictions).mean()

In [13]:
learn = Learner(dls, simple_net, opt_func=SGD,
                loss_func=mnist_loss)

In [14]:
learn.fit(45, 1)

epoch,train_loss,valid_loss,time
0,0.009807,0.113500,00:00
1,0.009807,0.113500,00:00
2,0.009807,0.113500,00:00
3,0.009807,0.113500,00:00
4,0.009807,0.113500,00:00
5,0.009807,0.113500,00:00
6,0.009807,0.113500,00:00
7,0.009807,0.113500,00:00
8,0.009807,0.113500,00:00
9,0.009807,0.113500,00:00
